In [11]:
from __future__ import division
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
import scipy.optimize as op 

from sklearn.model_selection import train_test_split,StratifiedShuffleSplit 
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn import metrics as metrices

%matplotlib inline

pd.options.display.float_format = '{:.2f}'.format

In [12]:
geisinger_afib_panda = pd.read_csv('geisinger_afib_patients.csv',low_memory=False)
geisinger_non_afib_panda = pd.read_csv('geisinger_non_afib_patients.csv',low_memory=False)

In [13]:
geisinger_afib_panda.count()

PT_ID                                                2744
PT_SEX                                               2744
PT_BIRTH_DT                                          2744
PT_DEATH_DT                                          1041
LAST_ACTIVE_DT                                       2744
                                                     ... 
CREATININE_MEDIAN_AFTER_INDEX                        2684
CREATININE_CLOSEST_BEFORE_RECUR_OR_LASTENC           2703
DT_CREATININE_CLOSEST_BEFORE_RECUR_OR_LASTENC        2703
CREATININE_CLOSEST_BEFORE_DEATH_OR_LASTENC           2703
DT_FOR_CREATININE_CLOSEST_BEFORE_DEATH_OR_LASTENC    2703
Length: 284, dtype: int64

In [14]:
#data for 2744 patients's first occurance of afib
data_afib_per_patient_panda = pd.read_csv('df_afib_no_duplicate_per_patient.csv')

#data for 1062 patients's whose first occurance of afib is before the stroke date
data_afib_diagnosed_before_storke = data_afib_per_patient_panda[data_afib_per_patient_panda['Occurance_in_years'] < 0]
data_afib_diagnosed_before_storke_at_index = data_afib_per_patient_panda[data_afib_per_patient_panda['Occurance_in_years'] == 0]
data_afib_diagnosed_after_storke_within_3_years =  data_afib_per_patient_panda[(data_afib_per_patient_panda['Occurance_in_years'] > 0) & (data_afib_per_patient_panda['Occurance_in_years'] <= 3)]
data_afib_diagnosed_other = data_afib_per_patient_panda[data_afib_per_patient_panda['Occurance_in_years'] > 3]

print(data_afib_diagnosed_before_storke['PT_ID'].nunique())
print(data_afib_diagnosed_before_storke_at_index['PT_ID'].nunique())
print(data_afib_diagnosed_after_storke_within_3_years['PT_ID'].nunique())
print(data_afib_diagnosed_other['PT_ID'].nunique())

1062
819
583
280


# $\color{BLUE}{\text{Patients having afib before stroke index}}$

In [15]:
geisinger_afib_panda_before_stroke = geisinger_afib_panda[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_before_storke['PT_ID'])]


In [16]:
comorbidity_data_afib_before_stroke = geisinger_afib_panda_before_stroke[['PT_ID','PT_SEX','PT_STATUS','FAM_HEART_HIST','FAM_STROKE_HIST','ATRIAL_FIB_PREINDEX','ATRIAL_FLUTTER_PREINDEX','AFIB_FLUTTER_PREINDEX','HYPERTENSION_PREINDEX','MI_PREINDEX','DIABETES_PREINDEX','DYSLIPIDEMIA_PREINDEX','CHF_PREINDEX','HYPERCOAG_STATES_PREINDEX','MILD_LIVER_DIS_PREINDEX','MOD_SEV_LIVER_DIS_PREINDEX','CHRONIC_LIVER_DIS_PREINDEX','CHRONIC_LUNG_DIS_PREINDEX','RHEUM_DIS_PREINDEX','CHRONIC_KIDNEY_DIS_PREINDEX','NEOPLASM_PREINDEX','PERI_VASC_DIS_PREINDEX','PFO_PREINDEX','PAST_ISCHEMIC_STROKE_PREINDEX','PAST_HEMORRHAGIC_STROKE_PREINDEX','ATRIAL_FIB_AT_INDEX','ATRIAL_FLUTTER_AT_INDEX','AFIB_FLUTTER_AT_INDEX','HYPERTENSION_AT_INDEX','MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX','HYPERCOAG_STATES_AT_INDEX','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX','CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX','CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX','PFO_AT_INDEX','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX']]

In [17]:
comorbidity_data_afib_before_stroke.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1062 entries, 1 to 2743
Data columns (total 45 columns):
PT_ID                               1062 non-null object
PT_SEX                              1062 non-null object
PT_STATUS                           1062 non-null object
FAM_HEART_HIST                      1062 non-null int64
FAM_STROKE_HIST                     1062 non-null int64
ATRIAL_FIB_PREINDEX                 1062 non-null int64
ATRIAL_FLUTTER_PREINDEX             1062 non-null int64
AFIB_FLUTTER_PREINDEX               1062 non-null int64
HYPERTENSION_PREINDEX               1062 non-null int64
MI_PREINDEX                         1062 non-null int64
DIABETES_PREINDEX                   1062 non-null int64
DYSLIPIDEMIA_PREINDEX               1062 non-null int64
CHF_PREINDEX                        1062 non-null int64
HYPERCOAG_STATES_PREINDEX           1062 non-null int64
MILD_LIVER_DIS_PREINDEX             1062 non-null int64
MOD_SEV_LIVER_DIS_PREINDEX          1062 non-null 

### Sex and Family History

In [18]:
SEX =comorbidity_data_afib_before_stroke.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
SEX['percentage (%)'] = SEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(SEX)
print('=======================================')

FAM_HEART_HIST =comorbidity_data_afib_before_stroke.groupby(['FAM_HEART_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_HEART_HIST['percentage (%)'] = FAM_HEART_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_HEART_HIST)

print('=======================================')
FAM_STROKE_HIST =comorbidity_data_afib_before_stroke.groupby(['FAM_STROKE_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_STROKE_HIST['percentage (%)'] = FAM_STROKE_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_STROKE_HIST)

   PT_SEX  Patients  percentage (%)
0  Female       556           52.35
1    Male       506           47.65
   FAM_HEART_HIST  Patients  percentage (%)
0               0       592           55.74
1               1       470           44.26
   FAM_STROKE_HIST  Patients  percentage (%)
0                0       900           84.75
1                1       162           15.25


### CoMorbidity PREINDEX

In [19]:
ATRIAL_FIB_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['ATRIAL_FIB_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_PREINDEX['percentage (%)'] = ATRIAL_FIB_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_PREINDEX)

print('=======================================')
ATRIAL_FLUTTER_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['ATRIAL_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_PREINDEX['percentage (%)'] = ATRIAL_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_PREINDEX)

print('=======================================')
AFIB_FLUTTER_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['AFIB_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_PREINDEX['percentage (%)'] = AFIB_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_PREINDEX)

print('=======================================')
HYPERTENSION_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['HYPERTENSION_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_PREINDEX['percentage (%)'] = HYPERTENSION_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_PREINDEX)

print('=======================================')
MI_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['MI_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_PREINDEX['percentage (%)'] = MI_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_PREINDEX)

print('=======================================')
DIABETES_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['DIABETES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_PREINDEX['percentage (%)'] = DIABETES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_PREINDEX)

print('=======================================')
DYSLIPIDEMIA_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['DYSLIPIDEMIA_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_PREINDEX['percentage (%)'] = DYSLIPIDEMIA_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_PREINDEX)

print('=======================================')
CHF_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['CHF_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_PREINDEX['percentage (%)'] = CHF_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_PREINDEX)

print('=======================================')
HYPERCOAG_STATES_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['HYPERCOAG_STATES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_PREINDEX['percentage (%)'] = HYPERCOAG_STATES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_PREINDEX)

print('=======================================')
MILD_LIVER_DIS_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['MILD_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_PREINDEX['percentage (%)'] = MILD_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_PREINDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['MOD_SEV_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_PREINDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LIVER_DIS_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['CHRONIC_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_PREINDEX['percentage (%)'] = CHRONIC_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LUNG_DIS_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['CHRONIC_LUNG_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_PREINDEX['percentage (%)'] = CHRONIC_LUNG_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_PREINDEX)

print('=======================================')
RHEUM_DIS_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['RHEUM_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_PREINDEX['percentage (%)'] = RHEUM_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_PREINDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['CHRONIC_KIDNEY_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_PREINDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_PREINDEX)

print('=======================================')
NEOPLASM_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['NEOPLASM_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_PREINDEX['percentage (%)'] = NEOPLASM_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_PREINDEX)

print('=======================================')
PERI_VASC_DIS_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['PERI_VASC_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_PREINDEX['percentage (%)'] = PERI_VASC_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_PREINDEX)

print('=======================================')
PFO_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['PFO_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_PREINDEX['percentage (%)'] = PFO_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_PREINDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['PAST_ISCHEMIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_PREINDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_PREINDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['PAST_HEMORRHAGIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_PREINDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_PREINDEX)

   ATRIAL_FIB_PREINDEX  Patients  percentage (%)
0                    1      1062          100.00
   ATRIAL_FLUTTER_PREINDEX  Patients  percentage (%)
0                        0       951           89.55
1                        1       111           10.45
   AFIB_FLUTTER_PREINDEX  Patients  percentage (%)
0                      1      1062          100.00
   HYPERTENSION_PREINDEX  Patients  percentage (%)
0                      0       162           15.25
1                      1       900           84.75
   MI_PREINDEX  Patients  percentage (%)
0            0       847           79.76
1            1       215           20.24
   DIABETES_PREINDEX  Patients  percentage (%)
0                  0       670           63.09
1                  1       392           36.91
   DYSLIPIDEMIA_PREINDEX  Patients  percentage (%)
0                      0       322           30.32
1                      1       740           69.68
   CHF_PREINDEX  Patients  percentage (%)
0             0       665    

### CoMorbidity AT INDEX

In [20]:
ATRIAL_FIB_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['ATRIAL_FIB_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_AT_INDEX['percentage (%)'] = ATRIAL_FIB_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_AT_INDEX)

print('=======================================')
ATRIAL_FLUTTER_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['ATRIAL_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_AT_INDEX['percentage (%)'] = ATRIAL_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_AT_INDEX)

print('=======================================')
AFIB_FLUTTER_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['AFIB_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_AT_INDEX['percentage (%)'] = AFIB_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_AT_INDEX)

print('=======================================')
HYPERTENSION_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['HYPERTENSION_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_AT_INDEX['percentage (%)'] = HYPERTENSION_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_AT_INDEX)

print('=======================================')
MI_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['MI_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_AT_INDEX['percentage (%)'] = MI_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_AT_INDEX)

print('=======================================')
DIABETES_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['DIABETES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_AT_INDEX['percentage (%)'] = DIABETES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_AT_INDEX)

print('=======================================')
DYSLIPIDEMIA_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['DYSLIPIDEMIA_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_AT_INDEX['percentage (%)'] = DYSLIPIDEMIA_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_AT_INDEX)

print('=======================================')
CHF_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['CHF_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_AT_INDEX['percentage (%)'] = CHF_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_AT_INDEX)

print('=======================================')
HYPERCOAG_STATES_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['HYPERCOAG_STATES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_AT_INDEX['percentage (%)'] = HYPERCOAG_STATES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_AT_INDEX)

print('=======================================')
MILD_LIVER_DIS_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['MILD_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_AT_INDEX['percentage (%)'] = MILD_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_AT_INDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['MOD_SEV_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_AT_INDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LIVER_DIS_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['CHRONIC_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LUNG_DIS_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['CHRONIC_LUNG_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LUNG_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_AT_INDEX)

print('=======================================')
RHEUM_DIS_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['RHEUM_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_AT_INDEX['percentage (%)'] = RHEUM_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_AT_INDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['CHRONIC_KIDNEY_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_AT_INDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_AT_INDEX)

print('=======================================')
NEOPLASM_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['NEOPLASM_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_AT_INDEX['percentage (%)'] = NEOPLASM_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_AT_INDEX)

print('=======================================')
PERI_VASC_DIS_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['PERI_VASC_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_AT_INDEX['percentage (%)'] = PERI_VASC_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_AT_INDEX)

print('=======================================')
PFO_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['PFO_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_AT_INDEX['percentage (%)'] = PFO_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_AT_INDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['PAST_ISCHEMIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_AT_INDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_AT_INDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['PAST_HEMORRHAGIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_AT_INDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_AT_INDEX)

   ATRIAL_FIB_AT_INDEX  Patients  percentage (%)
0                    1      1062          100.00
   ATRIAL_FLUTTER_AT_INDEX  Patients  percentage (%)
0                        0       940           88.51
1                        1       122           11.49
   AFIB_FLUTTER_AT_INDEX  Patients  percentage (%)
0                      1      1062          100.00
   HYPERTENSION_AT_INDEX  Patients  percentage (%)
0                      0       105            9.89
1                      1       957           90.11
   MI_AT_INDEX  Patients  percentage (%)
0            0       826           77.78
1            1       236           22.22
   DIABETES_AT_INDEX  Patients  percentage (%)
0                  0       647           60.92
1                  1       415           39.08
   DYSLIPIDEMIA_AT_INDEX  Patients  percentage (%)
0                      0       266           25.05
1                      1       796           74.95
   CHF_AT_INDEX  Patients  percentage (%)
0             0       634    

# $\color{BLUE}{\text{Patients having afib at stroke index}}$

In [21]:
geisinger_afib_panda_at_stroke = geisinger_afib_panda[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_before_storke_at_index['PT_ID'])]
comorbidity_data_afib_at_stroke = geisinger_afib_panda_at_stroke[['PT_ID','PT_SEX','PT_STATUS','FAM_HEART_HIST','FAM_STROKE_HIST','ATRIAL_FIB_PREINDEX','ATRIAL_FLUTTER_PREINDEX','AFIB_FLUTTER_PREINDEX','HYPERTENSION_PREINDEX','MI_PREINDEX','DIABETES_PREINDEX','DYSLIPIDEMIA_PREINDEX','CHF_PREINDEX','HYPERCOAG_STATES_PREINDEX','MILD_LIVER_DIS_PREINDEX','MOD_SEV_LIVER_DIS_PREINDEX','CHRONIC_LIVER_DIS_PREINDEX','CHRONIC_LUNG_DIS_PREINDEX','RHEUM_DIS_PREINDEX','CHRONIC_KIDNEY_DIS_PREINDEX','NEOPLASM_PREINDEX','PERI_VASC_DIS_PREINDEX','PFO_PREINDEX','PAST_ISCHEMIC_STROKE_PREINDEX','PAST_HEMORRHAGIC_STROKE_PREINDEX','ATRIAL_FIB_AT_INDEX','ATRIAL_FLUTTER_AT_INDEX','AFIB_FLUTTER_AT_INDEX','HYPERTENSION_AT_INDEX','MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX','HYPERCOAG_STATES_AT_INDEX','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX','CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX','CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX','PFO_AT_INDEX','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX']]
comorbidity_data_afib_at_stroke.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 819 entries, 0 to 2742
Data columns (total 45 columns):
PT_ID                               819 non-null object
PT_SEX                              819 non-null object
PT_STATUS                           819 non-null object
FAM_HEART_HIST                      819 non-null int64
FAM_STROKE_HIST                     819 non-null int64
ATRIAL_FIB_PREINDEX                 819 non-null int64
ATRIAL_FLUTTER_PREINDEX             819 non-null int64
AFIB_FLUTTER_PREINDEX               819 non-null int64
HYPERTENSION_PREINDEX               819 non-null int64
MI_PREINDEX                         819 non-null int64
DIABETES_PREINDEX                   819 non-null int64
DYSLIPIDEMIA_PREINDEX               819 non-null int64
CHF_PREINDEX                        819 non-null int64
HYPERCOAG_STATES_PREINDEX           819 non-null int64
MILD_LIVER_DIS_PREINDEX             819 non-null int64
MOD_SEV_LIVER_DIS_PREINDEX          819 non-null int64
CHRONIC_LIV

### Sex and Family History

In [22]:
SEX =comorbidity_data_afib_at_stroke.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
SEX['percentage (%)'] = SEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(SEX)
print('=======================================')

FAM_HEART_HIST =comorbidity_data_afib_at_stroke.groupby(['FAM_HEART_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_HEART_HIST['percentage (%)'] = FAM_HEART_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_HEART_HIST)

print('=======================================')
FAM_STROKE_HIST =comorbidity_data_afib_at_stroke.groupby(['FAM_STROKE_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_STROKE_HIST['percentage (%)'] = FAM_STROKE_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_STROKE_HIST)

   PT_SEX  Patients  percentage (%)
0  Female       433           52.87
1    Male       386           47.13
   FAM_HEART_HIST  Patients  percentage (%)
0               0       577           70.45
1               1       242           29.55
   FAM_STROKE_HIST  Patients  percentage (%)
0                0       720           87.91
1                1        99           12.09


### CoMorbidity PREINDEX

In [23]:
ATRIAL_FIB_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['ATRIAL_FIB_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_PREINDEX['percentage (%)'] = ATRIAL_FIB_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_PREINDEX)

print('=======================================')
ATRIAL_FLUTTER_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['ATRIAL_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_PREINDEX['percentage (%)'] = ATRIAL_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_PREINDEX)

print('=======================================')
AFIB_FLUTTER_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['AFIB_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_PREINDEX['percentage (%)'] = AFIB_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_PREINDEX)

print('=======================================')
HYPERTENSION_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['HYPERTENSION_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_PREINDEX['percentage (%)'] = HYPERTENSION_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_PREINDEX)

print('=======================================')
MI_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['MI_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_PREINDEX['percentage (%)'] = MI_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_PREINDEX)

print('=======================================')
DIABETES_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['DIABETES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_PREINDEX['percentage (%)'] = DIABETES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_PREINDEX)

print('=======================================')
DYSLIPIDEMIA_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['DYSLIPIDEMIA_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_PREINDEX['percentage (%)'] = DYSLIPIDEMIA_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_PREINDEX)

print('=======================================')
CHF_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['CHF_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_PREINDEX['percentage (%)'] = CHF_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_PREINDEX)

print('=======================================')
HYPERCOAG_STATES_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['HYPERCOAG_STATES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_PREINDEX['percentage (%)'] = HYPERCOAG_STATES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_PREINDEX)

print('=======================================')
MILD_LIVER_DIS_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['MILD_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_PREINDEX['percentage (%)'] = MILD_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_PREINDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['MOD_SEV_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_PREINDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LIVER_DIS_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['CHRONIC_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_PREINDEX['percentage (%)'] = CHRONIC_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LUNG_DIS_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['CHRONIC_LUNG_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_PREINDEX['percentage (%)'] = CHRONIC_LUNG_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_PREINDEX)

print('=======================================')
RHEUM_DIS_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['RHEUM_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_PREINDEX['percentage (%)'] = RHEUM_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_PREINDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['CHRONIC_KIDNEY_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_PREINDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_PREINDEX)

print('=======================================')
NEOPLASM_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['NEOPLASM_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_PREINDEX['percentage (%)'] = NEOPLASM_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_PREINDEX)

print('=======================================')
PERI_VASC_DIS_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['PERI_VASC_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_PREINDEX['percentage (%)'] = PERI_VASC_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_PREINDEX)

print('=======================================')
PFO_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['PFO_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_PREINDEX['percentage (%)'] = PFO_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_PREINDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['PAST_ISCHEMIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_PREINDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_PREINDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['PAST_HEMORRHAGIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_PREINDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_PREINDEX)

   ATRIAL_FIB_PREINDEX  Patients  percentage (%)
0                    0       819          100.00
   ATRIAL_FLUTTER_PREINDEX  Patients  percentage (%)
0                        0       810           98.90
1                        1         9            1.10
   AFIB_FLUTTER_PREINDEX  Patients  percentage (%)
0                      0       810           98.90
1                      1         9            1.10
   HYPERTENSION_PREINDEX  Patients  percentage (%)
0                      0       488           59.58
1                      1       331           40.42
   MI_PREINDEX  Patients  percentage (%)
0            0       771           94.14
1            1        48            5.86
   DIABETES_PREINDEX  Patients  percentage (%)
0                  0       683           83.39
1                  1       136           16.61
   DYSLIPIDEMIA_PREINDEX  Patients  percentage (%)
0                      0       545           66.54
1                      1       274           33.46
   CHF_PREINDEX  Pat

### CoMorbidity AT INDEX

In [24]:
ATRIAL_FIB_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['ATRIAL_FIB_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_AT_INDEX['percentage (%)'] = ATRIAL_FIB_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_AT_INDEX)

print('=======================================')
ATRIAL_FLUTTER_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['ATRIAL_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_AT_INDEX['percentage (%)'] = ATRIAL_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_AT_INDEX)

print('=======================================')
AFIB_FLUTTER_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['AFIB_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_AT_INDEX['percentage (%)'] = AFIB_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_AT_INDEX)

print('=======================================')
HYPERTENSION_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['HYPERTENSION_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_AT_INDEX['percentage (%)'] = HYPERTENSION_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_AT_INDEX)

print('=======================================')
MI_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['MI_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_AT_INDEX['percentage (%)'] = MI_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_AT_INDEX)

print('=======================================')
DIABETES_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['DIABETES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_AT_INDEX['percentage (%)'] = DIABETES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_AT_INDEX)

print('=======================================')
DYSLIPIDEMIA_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['DYSLIPIDEMIA_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_AT_INDEX['percentage (%)'] = DYSLIPIDEMIA_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_AT_INDEX)

print('=======================================')
CHF_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['CHF_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_AT_INDEX['percentage (%)'] = CHF_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_AT_INDEX)

print('=======================================')
HYPERCOAG_STATES_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['HYPERCOAG_STATES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_AT_INDEX['percentage (%)'] = HYPERCOAG_STATES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_AT_INDEX)

print('=======================================')
MILD_LIVER_DIS_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['MILD_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_AT_INDEX['percentage (%)'] = MILD_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_AT_INDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['MOD_SEV_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_AT_INDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LIVER_DIS_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['CHRONIC_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LUNG_DIS_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['CHRONIC_LUNG_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LUNG_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_AT_INDEX)

print('=======================================')
RHEUM_DIS_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['RHEUM_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_AT_INDEX['percentage (%)'] = RHEUM_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_AT_INDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['CHRONIC_KIDNEY_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_AT_INDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_AT_INDEX)

print('=======================================')
NEOPLASM_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['NEOPLASM_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_AT_INDEX['percentage (%)'] = NEOPLASM_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_AT_INDEX)

print('=======================================')
PERI_VASC_DIS_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['PERI_VASC_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_AT_INDEX['percentage (%)'] = PERI_VASC_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_AT_INDEX)

print('=======================================')
PFO_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['PFO_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_AT_INDEX['percentage (%)'] = PFO_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_AT_INDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['PAST_ISCHEMIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_AT_INDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_AT_INDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['PAST_HEMORRHAGIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_AT_INDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_AT_INDEX)

   ATRIAL_FIB_AT_INDEX  Patients  percentage (%)
0                    1       819          100.00
   ATRIAL_FLUTTER_AT_INDEX  Patients  percentage (%)
0                        0       757           92.43
1                        1        62            7.57
   AFIB_FLUTTER_AT_INDEX  Patients  percentage (%)
0                      1       819          100.00
   HYPERTENSION_AT_INDEX  Patients  percentage (%)
0                      0       129           15.75
1                      1       690           84.25
   MI_AT_INDEX  Patients  percentage (%)
0            0       735           89.74
1            1        84           10.26
   DIABETES_AT_INDEX  Patients  percentage (%)
0                  0       568           69.35
1                  1       251           30.65
   DYSLIPIDEMIA_AT_INDEX  Patients  percentage (%)
0                      0       330           40.29
1                      1       489           59.71
   CHF_AT_INDEX  Patients  percentage (%)
0             0       676    

# $\color{BLUE}{\text{Patients having afib within 3 years after stroke index}}$

In [25]:
geisinger_afib_panda_after_storke_within_3_years = geisinger_afib_panda[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_after_storke_within_3_years['PT_ID'])]
comorbidity_data_afib_after_storke_within_3_years = geisinger_afib_panda_after_storke_within_3_years[['PT_ID','PT_SEX','PT_STATUS','FAM_HEART_HIST','FAM_STROKE_HIST','ATRIAL_FIB_PREINDEX','ATRIAL_FLUTTER_PREINDEX','AFIB_FLUTTER_PREINDEX','HYPERTENSION_PREINDEX','MI_PREINDEX','DIABETES_PREINDEX','DYSLIPIDEMIA_PREINDEX','CHF_PREINDEX','HYPERCOAG_STATES_PREINDEX','MILD_LIVER_DIS_PREINDEX','MOD_SEV_LIVER_DIS_PREINDEX','CHRONIC_LIVER_DIS_PREINDEX','CHRONIC_LUNG_DIS_PREINDEX','RHEUM_DIS_PREINDEX','CHRONIC_KIDNEY_DIS_PREINDEX','NEOPLASM_PREINDEX','PERI_VASC_DIS_PREINDEX','PFO_PREINDEX','PAST_ISCHEMIC_STROKE_PREINDEX','PAST_HEMORRHAGIC_STROKE_PREINDEX','ATRIAL_FIB_AT_INDEX','ATRIAL_FLUTTER_AT_INDEX','AFIB_FLUTTER_AT_INDEX','HYPERTENSION_AT_INDEX','MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX','HYPERCOAG_STATES_AT_INDEX','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX','CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX','CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX','PFO_AT_INDEX','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX']]
comorbidity_data_afib_after_storke_within_3_years.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 583 entries, 4 to 2732
Data columns (total 45 columns):
PT_ID                               583 non-null object
PT_SEX                              583 non-null object
PT_STATUS                           583 non-null object
FAM_HEART_HIST                      583 non-null int64
FAM_STROKE_HIST                     583 non-null int64
ATRIAL_FIB_PREINDEX                 583 non-null int64
ATRIAL_FLUTTER_PREINDEX             583 non-null int64
AFIB_FLUTTER_PREINDEX               583 non-null int64
HYPERTENSION_PREINDEX               583 non-null int64
MI_PREINDEX                         583 non-null int64
DIABETES_PREINDEX                   583 non-null int64
DYSLIPIDEMIA_PREINDEX               583 non-null int64
CHF_PREINDEX                        583 non-null int64
HYPERCOAG_STATES_PREINDEX           583 non-null int64
MILD_LIVER_DIS_PREINDEX             583 non-null int64
MOD_SEV_LIVER_DIS_PREINDEX          583 non-null int64
CHRONIC_LIV

### Sex and Family History

In [26]:
SEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
SEX['percentage (%)'] = SEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(SEX)
print('=======================================')

FAM_HEART_HIST =comorbidity_data_afib_after_storke_within_3_years.groupby(['FAM_HEART_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_HEART_HIST['percentage (%)'] = FAM_HEART_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_HEART_HIST)

print('=======================================')
FAM_STROKE_HIST =comorbidity_data_afib_after_storke_within_3_years.groupby(['FAM_STROKE_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_STROKE_HIST['percentage (%)'] = FAM_STROKE_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_STROKE_HIST)

   PT_SEX  Patients  percentage (%)
0  Female       302           51.80
1    Male       281           48.20
   FAM_HEART_HIST  Patients  percentage (%)
0               0       370           63.46
1               1       213           36.54
   FAM_STROKE_HIST  Patients  percentage (%)
0                0       502           86.11
1                1        81           13.89


### CoMorbidity PREINDEX

In [27]:
ATRIAL_FIB_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['ATRIAL_FIB_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_PREINDEX['percentage (%)'] = ATRIAL_FIB_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_PREINDEX)

print('=======================================')
ATRIAL_FLUTTER_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['ATRIAL_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_PREINDEX['percentage (%)'] = ATRIAL_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_PREINDEX)

print('=======================================')
AFIB_FLUTTER_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['AFIB_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_PREINDEX['percentage (%)'] = AFIB_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_PREINDEX)

print('=======================================')
HYPERTENSION_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['HYPERTENSION_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_PREINDEX['percentage (%)'] = HYPERTENSION_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_PREINDEX)

print('=======================================')
MI_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['MI_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_PREINDEX['percentage (%)'] = MI_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_PREINDEX)

print('=======================================')
DIABETES_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['DIABETES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_PREINDEX['percentage (%)'] = DIABETES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_PREINDEX)

print('=======================================')
DYSLIPIDEMIA_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['DYSLIPIDEMIA_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_PREINDEX['percentage (%)'] = DYSLIPIDEMIA_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_PREINDEX)

print('=======================================')
CHF_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHF_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_PREINDEX['percentage (%)'] = CHF_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_PREINDEX)

print('=======================================')
HYPERCOAG_STATES_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['HYPERCOAG_STATES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_PREINDEX['percentage (%)'] = HYPERCOAG_STATES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_PREINDEX)

print('=======================================')
MILD_LIVER_DIS_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['MILD_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_PREINDEX['percentage (%)'] = MILD_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_PREINDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['MOD_SEV_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_PREINDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LIVER_DIS_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHRONIC_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_PREINDEX['percentage (%)'] = CHRONIC_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LUNG_DIS_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHRONIC_LUNG_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_PREINDEX['percentage (%)'] = CHRONIC_LUNG_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_PREINDEX)

print('=======================================')
RHEUM_DIS_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['RHEUM_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_PREINDEX['percentage (%)'] = RHEUM_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_PREINDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHRONIC_KIDNEY_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_PREINDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_PREINDEX)

print('=======================================')
NEOPLASM_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['NEOPLASM_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_PREINDEX['percentage (%)'] = NEOPLASM_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_PREINDEX)

print('=======================================')
PERI_VASC_DIS_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PERI_VASC_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_PREINDEX['percentage (%)'] = PERI_VASC_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_PREINDEX)

print('=======================================')
PFO_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PFO_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_PREINDEX['percentage (%)'] = PFO_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_PREINDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PAST_ISCHEMIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_PREINDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_PREINDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PAST_HEMORRHAGIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_PREINDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_PREINDEX)

   ATRIAL_FIB_PREINDEX  Patients  percentage (%)
0                    0       583          100.00
   ATRIAL_FLUTTER_PREINDEX  Patients  percentage (%)
0                        0       577           98.97
1                        1         6            1.03
   AFIB_FLUTTER_PREINDEX  Patients  percentage (%)
0                      0       577           98.97
1                      1         6            1.03
   HYPERTENSION_PREINDEX  Patients  percentage (%)
0                      0       270           46.31
1                      1       313           53.69
   MI_PREINDEX  Patients  percentage (%)
0            0       536           91.94
1            1        47            8.06
   DIABETES_PREINDEX  Patients  percentage (%)
0                  0       420           72.04
1                  1       163           27.96
   DYSLIPIDEMIA_PREINDEX  Patients  percentage (%)
0                      0       321           55.06
1                      1       262           44.94
   CHF_PREINDEX  Pat

### CoMorbidity AT INDEX

In [28]:
ATRIAL_FIB_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['ATRIAL_FIB_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_AT_INDEX['percentage (%)'] = ATRIAL_FIB_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_AT_INDEX)

print('=======================================')
ATRIAL_FLUTTER_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['ATRIAL_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_AT_INDEX['percentage (%)'] = ATRIAL_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_AT_INDEX)

print('=======================================')
AFIB_FLUTTER_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['AFIB_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_AT_INDEX['percentage (%)'] = AFIB_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_AT_INDEX)

print('=======================================')
HYPERTENSION_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['HYPERTENSION_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_AT_INDEX['percentage (%)'] = HYPERTENSION_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_AT_INDEX)

print('=======================================')
MI_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['MI_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_AT_INDEX['percentage (%)'] = MI_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_AT_INDEX)

print('=======================================')
DIABETES_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['DIABETES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_AT_INDEX['percentage (%)'] = DIABETES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_AT_INDEX)

print('=======================================')
DYSLIPIDEMIA_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['DYSLIPIDEMIA_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_AT_INDEX['percentage (%)'] = DYSLIPIDEMIA_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_AT_INDEX)

print('=======================================')
CHF_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHF_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_AT_INDEX['percentage (%)'] = CHF_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_AT_INDEX)

print('=======================================')
HYPERCOAG_STATES_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['HYPERCOAG_STATES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_AT_INDEX['percentage (%)'] = HYPERCOAG_STATES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_AT_INDEX)

print('=======================================')
MILD_LIVER_DIS_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['MILD_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_AT_INDEX['percentage (%)'] = MILD_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_AT_INDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['MOD_SEV_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_AT_INDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LIVER_DIS_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHRONIC_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LUNG_DIS_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHRONIC_LUNG_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LUNG_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_AT_INDEX)

print('=======================================')
RHEUM_DIS_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['RHEUM_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_AT_INDEX['percentage (%)'] = RHEUM_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_AT_INDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHRONIC_KIDNEY_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_AT_INDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_AT_INDEX)

print('=======================================')
NEOPLASM_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['NEOPLASM_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_AT_INDEX['percentage (%)'] = NEOPLASM_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_AT_INDEX)

print('=======================================')
PERI_VASC_DIS_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PERI_VASC_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_AT_INDEX['percentage (%)'] = PERI_VASC_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_AT_INDEX)

print('=======================================')
PFO_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PFO_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_AT_INDEX['percentage (%)'] = PFO_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_AT_INDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PAST_ISCHEMIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_AT_INDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_AT_INDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PAST_HEMORRHAGIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_AT_INDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_AT_INDEX)

   ATRIAL_FIB_AT_INDEX  Patients  percentage (%)
0                    0       583          100.00
   ATRIAL_FLUTTER_AT_INDEX  Patients  percentage (%)
0                        0       561           96.23
1                        1        22            3.77
   AFIB_FLUTTER_AT_INDEX  Patients  percentage (%)
0                      0       561           96.23
1                      1        22            3.77
   HYPERTENSION_AT_INDEX  Patients  percentage (%)
0                      0       152           26.07
1                      1       431           73.93
   MI_AT_INDEX  Patients  percentage (%)
0            0       514           88.16
1            1        69           11.84
   DIABETES_AT_INDEX  Patients  percentage (%)
0                  0       383           65.69
1                  1       200           34.31
   DYSLIPIDEMIA_AT_INDEX  Patients  percentage (%)
0                      0       232           39.79
1                      1       351           60.21
   CHF_AT_INDEX  Pat

# $\color{BLUE}{\text{Patients having afib 3 years after stroke index}}$

In [29]:
geisinger_afib_panda_other = geisinger_afib_panda[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_other['PT_ID'])]
comorbidity_data_afib_other = geisinger_afib_panda_other[['PT_ID','PT_SEX','PT_STATUS','FAM_HEART_HIST','FAM_STROKE_HIST','ATRIAL_FIB_PREINDEX','ATRIAL_FLUTTER_PREINDEX','AFIB_FLUTTER_PREINDEX','HYPERTENSION_PREINDEX','MI_PREINDEX','DIABETES_PREINDEX','DYSLIPIDEMIA_PREINDEX','CHF_PREINDEX','HYPERCOAG_STATES_PREINDEX','MILD_LIVER_DIS_PREINDEX','MOD_SEV_LIVER_DIS_PREINDEX','CHRONIC_LIVER_DIS_PREINDEX','CHRONIC_LUNG_DIS_PREINDEX','RHEUM_DIS_PREINDEX','CHRONIC_KIDNEY_DIS_PREINDEX','NEOPLASM_PREINDEX','PERI_VASC_DIS_PREINDEX','PFO_PREINDEX','PAST_ISCHEMIC_STROKE_PREINDEX','PAST_HEMORRHAGIC_STROKE_PREINDEX','ATRIAL_FIB_AT_INDEX','ATRIAL_FLUTTER_AT_INDEX','AFIB_FLUTTER_AT_INDEX','HYPERTENSION_AT_INDEX','MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX','HYPERCOAG_STATES_AT_INDEX','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX','CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX','CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX','PFO_AT_INDEX','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX']]
comorbidity_data_afib_other.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280 entries, 2 to 2740
Data columns (total 45 columns):
PT_ID                               280 non-null object
PT_SEX                              280 non-null object
PT_STATUS                           280 non-null object
FAM_HEART_HIST                      280 non-null int64
FAM_STROKE_HIST                     280 non-null int64
ATRIAL_FIB_PREINDEX                 280 non-null int64
ATRIAL_FLUTTER_PREINDEX             280 non-null int64
AFIB_FLUTTER_PREINDEX               280 non-null int64
HYPERTENSION_PREINDEX               280 non-null int64
MI_PREINDEX                         280 non-null int64
DIABETES_PREINDEX                   280 non-null int64
DYSLIPIDEMIA_PREINDEX               280 non-null int64
CHF_PREINDEX                        280 non-null int64
HYPERCOAG_STATES_PREINDEX           280 non-null int64
MILD_LIVER_DIS_PREINDEX             280 non-null int64
MOD_SEV_LIVER_DIS_PREINDEX          280 non-null int64
CHRONIC_LIV

### Sex and Family History

In [30]:
SEX =comorbidity_data_afib_other.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
SEX['percentage (%)'] = SEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(SEX)

print('=======================================')
FAM_HEART_HIST =comorbidity_data_afib_other.groupby(['FAM_HEART_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_HEART_HIST['percentage (%)'] = FAM_HEART_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_HEART_HIST)

print('=======================================')
FAM_STROKE_HIST =comorbidity_data_afib_other.groupby(['FAM_STROKE_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_STROKE_HIST['percentage (%)'] = FAM_STROKE_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_STROKE_HIST)

   PT_SEX  Patients  percentage (%)
0  Female       146           52.14
1    Male       134           47.86
   FAM_HEART_HIST  Patients  percentage (%)
0               0       159           56.79
1               1       121           43.21
   FAM_STROKE_HIST  Patients  percentage (%)
0                0       227           81.07
1                1        53           18.93


### CoMorbidity PREINDEX

In [31]:
ATRIAL_FIB_PREINDEX =comorbidity_data_afib_other.groupby(['ATRIAL_FIB_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_PREINDEX['percentage (%)'] = ATRIAL_FIB_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_PREINDEX)

print('=======================================')
ATRIAL_FLUTTER_PREINDEX =comorbidity_data_afib_other.groupby(['ATRIAL_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_PREINDEX['percentage (%)'] = ATRIAL_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_PREINDEX)

print('=======================================')
AFIB_FLUTTER_PREINDEX =comorbidity_data_afib_other.groupby(['AFIB_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_PREINDEX['percentage (%)'] = AFIB_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_PREINDEX)

print('=======================================')
HYPERTENSION_PREINDEX =comorbidity_data_afib_other.groupby(['HYPERTENSION_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_PREINDEX['percentage (%)'] = HYPERTENSION_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_PREINDEX)

print('=======================================')
MI_PREINDEX =comorbidity_data_afib_other.groupby(['MI_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_PREINDEX['percentage (%)'] = MI_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_PREINDEX)

print('=======================================')
DIABETES_PREINDEX =comorbidity_data_afib_other.groupby(['DIABETES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_PREINDEX['percentage (%)'] = DIABETES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_PREINDEX)

print('=======================================')
DYSLIPIDEMIA_PREINDEX =comorbidity_data_afib_other.groupby(['DYSLIPIDEMIA_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_PREINDEX['percentage (%)'] = DYSLIPIDEMIA_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_PREINDEX)

print('=======================================')
CHF_PREINDEX =comorbidity_data_afib_other.groupby(['CHF_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_PREINDEX['percentage (%)'] = CHF_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_PREINDEX)

print('=======================================')
HYPERCOAG_STATES_PREINDEX =comorbidity_data_afib_other.groupby(['HYPERCOAG_STATES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_PREINDEX['percentage (%)'] = HYPERCOAG_STATES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_PREINDEX)

print('=======================================')
MILD_LIVER_DIS_PREINDEX =comorbidity_data_afib_other.groupby(['MILD_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_PREINDEX['percentage (%)'] = MILD_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_PREINDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_PREINDEX =comorbidity_data_afib_other.groupby(['MOD_SEV_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_PREINDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LIVER_DIS_PREINDEX =comorbidity_data_afib_other.groupby(['CHRONIC_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_PREINDEX['percentage (%)'] = CHRONIC_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LUNG_DIS_PREINDEX =comorbidity_data_afib_other.groupby(['CHRONIC_LUNG_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_PREINDEX['percentage (%)'] = CHRONIC_LUNG_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_PREINDEX)

print('=======================================')
RHEUM_DIS_PREINDEX =comorbidity_data_afib_other.groupby(['RHEUM_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_PREINDEX['percentage (%)'] = RHEUM_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_PREINDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_PREINDEX =comorbidity_data_afib_other.groupby(['CHRONIC_KIDNEY_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_PREINDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_PREINDEX)

print('=======================================')
NEOPLASM_PREINDEX =comorbidity_data_afib_other.groupby(['NEOPLASM_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_PREINDEX['percentage (%)'] = NEOPLASM_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_PREINDEX)

print('=======================================')
PERI_VASC_DIS_PREINDEX =comorbidity_data_afib_other.groupby(['PERI_VASC_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_PREINDEX['percentage (%)'] = PERI_VASC_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_PREINDEX)

print('=======================================')
PFO_PREINDEX =comorbidity_data_afib_other.groupby(['PFO_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_PREINDEX['percentage (%)'] = PFO_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_PREINDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_PREINDEX =comorbidity_data_afib_other.groupby(['PAST_ISCHEMIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_PREINDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_PREINDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_PREINDEX =comorbidity_data_afib_other.groupby(['PAST_HEMORRHAGIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_PREINDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_PREINDEX)

   ATRIAL_FIB_PREINDEX  Patients  percentage (%)
0                    0       280          100.00
   ATRIAL_FLUTTER_PREINDEX  Patients  percentage (%)
0                        0       280          100.00
   AFIB_FLUTTER_PREINDEX  Patients  percentage (%)
0                      0       280          100.00
   HYPERTENSION_PREINDEX  Patients  percentage (%)
0                      0       139           49.64
1                      1       141           50.36
   MI_PREINDEX  Patients  percentage (%)
0            0       266           95.00
1            1        14            5.00
   DIABETES_PREINDEX  Patients  percentage (%)
0                  0       205           73.21
1                  1        75           26.79
   DYSLIPIDEMIA_PREINDEX  Patients  percentage (%)
0                      0       160           57.14
1                      1       120           42.86
   CHF_PREINDEX  Patients  percentage (%)
0             0       261           93.21
1             1        19            6.7

### CoMorbidity AT INDEX

In [32]:
ATRIAL_FIB_AT_INDEX =comorbidity_data_afib_other.groupby(['ATRIAL_FIB_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_AT_INDEX['percentage (%)'] = ATRIAL_FIB_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_AT_INDEX)

print('=======================================')
ATRIAL_FLUTTER_AT_INDEX =comorbidity_data_afib_other.groupby(['ATRIAL_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_AT_INDEX['percentage (%)'] = ATRIAL_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_AT_INDEX)

print('=======================================')
AFIB_FLUTTER_AT_INDEX =comorbidity_data_afib_other.groupby(['AFIB_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_AT_INDEX['percentage (%)'] = AFIB_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_AT_INDEX)

print('=======================================')
HYPERTENSION_AT_INDEX =comorbidity_data_afib_other.groupby(['HYPERTENSION_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_AT_INDEX['percentage (%)'] = HYPERTENSION_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_AT_INDEX)

print('=======================================')
MI_AT_INDEX =comorbidity_data_afib_other.groupby(['MI_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_AT_INDEX['percentage (%)'] = MI_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_AT_INDEX)

print('=======================================')
DIABETES_AT_INDEX =comorbidity_data_afib_other.groupby(['DIABETES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_AT_INDEX['percentage (%)'] = DIABETES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_AT_INDEX)

print('=======================================')
DYSLIPIDEMIA_AT_INDEX =comorbidity_data_afib_other.groupby(['DYSLIPIDEMIA_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_AT_INDEX['percentage (%)'] = DYSLIPIDEMIA_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_AT_INDEX)

print('=======================================')
CHF_AT_INDEX =comorbidity_data_afib_other.groupby(['CHF_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_AT_INDEX['percentage (%)'] = CHF_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_AT_INDEX)

print('=======================================')
HYPERCOAG_STATES_AT_INDEX =comorbidity_data_afib_other.groupby(['HYPERCOAG_STATES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_AT_INDEX['percentage (%)'] = HYPERCOAG_STATES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_AT_INDEX)

print('=======================================')
MILD_LIVER_DIS_AT_INDEX =comorbidity_data_afib_other.groupby(['MILD_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_AT_INDEX['percentage (%)'] = MILD_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_AT_INDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_AT_INDEX =comorbidity_data_afib_other.groupby(['MOD_SEV_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_AT_INDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LIVER_DIS_AT_INDEX =comorbidity_data_afib_other.groupby(['CHRONIC_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LUNG_DIS_AT_INDEX =comorbidity_data_afib_other.groupby(['CHRONIC_LUNG_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LUNG_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_AT_INDEX)

print('=======================================')
RHEUM_DIS_AT_INDEX =comorbidity_data_afib_other.groupby(['RHEUM_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_AT_INDEX['percentage (%)'] = RHEUM_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_AT_INDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_AT_INDEX =comorbidity_data_afib_other.groupby(['CHRONIC_KIDNEY_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_AT_INDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_AT_INDEX)

print('=======================================')
NEOPLASM_AT_INDEX =comorbidity_data_afib_other.groupby(['NEOPLASM_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_AT_INDEX['percentage (%)'] = NEOPLASM_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_AT_INDEX)

print('=======================================')
PERI_VASC_DIS_AT_INDEX =comorbidity_data_afib_other.groupby(['PERI_VASC_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_AT_INDEX['percentage (%)'] = PERI_VASC_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_AT_INDEX)

print('=======================================')
PFO_AT_INDEX =comorbidity_data_afib_other.groupby(['PFO_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_AT_INDEX['percentage (%)'] = PFO_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_AT_INDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_AT_INDEX =comorbidity_data_afib_other.groupby(['PAST_ISCHEMIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_AT_INDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_AT_INDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_AT_INDEX =comorbidity_data_afib_other.groupby(['PAST_HEMORRHAGIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_AT_INDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_AT_INDEX)

   ATRIAL_FIB_AT_INDEX  Patients  percentage (%)
0                    0       280          100.00
   ATRIAL_FLUTTER_AT_INDEX  Patients  percentage (%)
0                        0       280          100.00
   AFIB_FLUTTER_AT_INDEX  Patients  percentage (%)
0                      0       280          100.00
   HYPERTENSION_AT_INDEX  Patients  percentage (%)
0                      0        80           28.57
1                      1       200           71.43
   MI_AT_INDEX  Patients  percentage (%)
0            0       258           92.14
1            1        22            7.86
   DIABETES_AT_INDEX  Patients  percentage (%)
0                  0       181           64.64
1                  1        99           35.36
   DYSLIPIDEMIA_AT_INDEX  Patients  percentage (%)
0                      0       102           36.43
1                      1       178           63.57
   CHF_AT_INDEX  Patients  percentage (%)
0             0       258           92.14
1             1        22            7.8

# $\color{RED}{\text{Patients having no Afib at all}}$

In [33]:
comorbidity_data_non_afib = geisinger_non_afib_panda[['PT_ID','PT_SEX','PT_STATUS','FAM_HEART_HIST','FAM_STROKE_HIST','ATRIAL_FIB_PREINDEX','ATRIAL_FLUTTER_PREINDEX','AFIB_FLUTTER_PREINDEX','HYPERTENSION_PREINDEX','MI_PREINDEX','DIABETES_PREINDEX','DYSLIPIDEMIA_PREINDEX','CHF_PREINDEX','HYPERCOAG_STATES_PREINDEX','MILD_LIVER_DIS_PREINDEX','MOD_SEV_LIVER_DIS_PREINDEX','CHRONIC_LIVER_DIS_PREINDEX','CHRONIC_LUNG_DIS_PREINDEX','RHEUM_DIS_PREINDEX','CHRONIC_KIDNEY_DIS_PREINDEX','NEOPLASM_PREINDEX','PERI_VASC_DIS_PREINDEX','PFO_PREINDEX','PAST_ISCHEMIC_STROKE_PREINDEX','PAST_HEMORRHAGIC_STROKE_PREINDEX','ATRIAL_FIB_AT_INDEX','ATRIAL_FLUTTER_AT_INDEX','AFIB_FLUTTER_AT_INDEX','HYPERTENSION_AT_INDEX','MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX','HYPERCOAG_STATES_AT_INDEX','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX','CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX','CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX','PFO_AT_INDEX','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX']]
comorbidity_data_non_afib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6185 entries, 0 to 6184
Data columns (total 45 columns):
PT_ID                               6185 non-null object
PT_SEX                              6185 non-null object
PT_STATUS                           6185 non-null object
FAM_HEART_HIST                      6185 non-null int64
FAM_STROKE_HIST                     6185 non-null int64
ATRIAL_FIB_PREINDEX                 6185 non-null int64
ATRIAL_FLUTTER_PREINDEX             6185 non-null int64
AFIB_FLUTTER_PREINDEX               6185 non-null int64
HYPERTENSION_PREINDEX               6185 non-null int64
MI_PREINDEX                         6185 non-null int64
DIABETES_PREINDEX                   6185 non-null int64
DYSLIPIDEMIA_PREINDEX               6185 non-null int64
CHF_PREINDEX                        6185 non-null int64
HYPERCOAG_STATES_PREINDEX           6185 non-null int64
MILD_LIVER_DIS_PREINDEX             6185 non-null int64
MOD_SEV_LIVER_DIS_PREINDEX          6185 non-null 

### Sex and Family History

In [34]:
SEX =comorbidity_data_non_afib.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
SEX['percentage (%)'] = SEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(SEX)

print('=======================================')
FAM_HEART_HIST =comorbidity_data_non_afib.groupby(['FAM_HEART_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_HEART_HIST['percentage (%)'] = FAM_HEART_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_HEART_HIST)

print('=======================================')
FAM_STROKE_HIST =comorbidity_data_non_afib.groupby(['FAM_STROKE_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_STROKE_HIST['percentage (%)'] = FAM_STROKE_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_STROKE_HIST)

   PT_SEX  Patients  percentage (%)
0  Female      2882           46.60
1    Male      3303           53.40
   FAM_HEART_HIST  Patients  percentage (%)
0               0      4063           65.69
1               1      2122           34.31
   FAM_STROKE_HIST  Patients  percentage (%)
0                0      5376           86.92
1                1       809           13.08


### CoMorbidity PREINDEX

In [35]:
ATRIAL_FIB_PREINDEX =comorbidity_data_non_afib.groupby(['ATRIAL_FIB_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_PREINDEX['percentage (%)'] = ATRIAL_FIB_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_PREINDEX)

print('=======================================')
ATRIAL_FLUTTER_PREINDEX =comorbidity_data_non_afib.groupby(['ATRIAL_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_PREINDEX['percentage (%)'] = ATRIAL_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_PREINDEX)

print('=======================================')
AFIB_FLUTTER_PREINDEX =comorbidity_data_non_afib.groupby(['AFIB_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_PREINDEX['percentage (%)'] = AFIB_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_PREINDEX)

print('=======================================')
HYPERTENSION_PREINDEX =comorbidity_data_non_afib.groupby(['HYPERTENSION_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_PREINDEX['percentage (%)'] = HYPERTENSION_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_PREINDEX)

print('=======================================')
MI_PREINDEX =comorbidity_data_non_afib.groupby(['MI_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_PREINDEX['percentage (%)'] = MI_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_PREINDEX)

print('=======================================')
DIABETES_PREINDEX =comorbidity_data_non_afib.groupby(['DIABETES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_PREINDEX['percentage (%)'] = DIABETES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_PREINDEX)

print('=======================================')
DYSLIPIDEMIA_PREINDEX =comorbidity_data_non_afib.groupby(['DYSLIPIDEMIA_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_PREINDEX['percentage (%)'] = DYSLIPIDEMIA_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_PREINDEX)

print('=======================================')
CHF_PREINDEX =comorbidity_data_non_afib.groupby(['CHF_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_PREINDEX['percentage (%)'] = CHF_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_PREINDEX)

print('=======================================')
HYPERCOAG_STATES_PREINDEX =comorbidity_data_non_afib.groupby(['HYPERCOAG_STATES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_PREINDEX['percentage (%)'] = HYPERCOAG_STATES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_PREINDEX)

print('=======================================')
MILD_LIVER_DIS_PREINDEX =comorbidity_data_non_afib.groupby(['MILD_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_PREINDEX['percentage (%)'] = MILD_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_PREINDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_PREINDEX =comorbidity_data_non_afib.groupby(['MOD_SEV_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_PREINDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LIVER_DIS_PREINDEX =comorbidity_data_non_afib.groupby(['CHRONIC_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_PREINDEX['percentage (%)'] = CHRONIC_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LUNG_DIS_PREINDEX =comorbidity_data_non_afib.groupby(['CHRONIC_LUNG_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_PREINDEX['percentage (%)'] = CHRONIC_LUNG_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_PREINDEX)

print('=======================================')
RHEUM_DIS_PREINDEX =comorbidity_data_non_afib.groupby(['RHEUM_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_PREINDEX['percentage (%)'] = RHEUM_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_PREINDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_PREINDEX =comorbidity_data_non_afib.groupby(['CHRONIC_KIDNEY_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_PREINDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_PREINDEX)

print('=======================================')
NEOPLASM_PREINDEX =comorbidity_data_non_afib.groupby(['NEOPLASM_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_PREINDEX['percentage (%)'] = NEOPLASM_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_PREINDEX)

print('=======================================')
PERI_VASC_DIS_PREINDEX =comorbidity_data_non_afib.groupby(['PERI_VASC_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_PREINDEX['percentage (%)'] = PERI_VASC_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_PREINDEX)

print('=======================================')
PFO_PREINDEX =comorbidity_data_non_afib.groupby(['PFO_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_PREINDEX['percentage (%)'] = PFO_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_PREINDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_PREINDEX =comorbidity_data_non_afib.groupby(['PAST_ISCHEMIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_PREINDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_PREINDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_PREINDEX =comorbidity_data_non_afib.groupby(['PAST_HEMORRHAGIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_PREINDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_PREINDEX)

   ATRIAL_FIB_PREINDEX  Patients  percentage (%)
0                    0      6185          100.00
   ATRIAL_FLUTTER_PREINDEX  Patients  percentage (%)
0                        0      6172           99.79
1                        1        13            0.21
   AFIB_FLUTTER_PREINDEX  Patients  percentage (%)
0                      0      6172           99.79
1                      1        13            0.21
   HYPERTENSION_PREINDEX  Patients  percentage (%)
0                      0      3431           55.47
1                      1      2754           44.53
   MI_PREINDEX  Patients  percentage (%)
0            0      5757           93.08
1            1       428            6.92
   DIABETES_PREINDEX  Patients  percentage (%)
0                  0      4836           78.19
1                  1      1349           21.81
   DYSLIPIDEMIA_PREINDEX  Patients  percentage (%)
0                      0      3773           61.00
1                      1      2412           39.00
   CHF_PREINDEX  Pat

### CoMorbidity AT INDEX

In [36]:
ATRIAL_FIB_AT_INDEX =comorbidity_data_non_afib.groupby(['ATRIAL_FIB_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_AT_INDEX['percentage (%)'] = ATRIAL_FIB_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_AT_INDEX)

print('=======================================')
ATRIAL_FLUTTER_AT_INDEX =comorbidity_data_non_afib.groupby(['ATRIAL_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_AT_INDEX['percentage (%)'] = ATRIAL_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_AT_INDEX)

print('=======================================')
AFIB_FLUTTER_AT_INDEX =comorbidity_data_non_afib.groupby(['AFIB_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_AT_INDEX['percentage (%)'] = AFIB_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_AT_INDEX)

print('=======================================')
HYPERTENSION_AT_INDEX =comorbidity_data_non_afib.groupby(['HYPERTENSION_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_AT_INDEX['percentage (%)'] = HYPERTENSION_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_AT_INDEX)

print('=======================================')
MI_AT_INDEX =comorbidity_data_non_afib.groupby(['MI_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_AT_INDEX['percentage (%)'] = MI_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_AT_INDEX)

print('=======================================')
DIABETES_AT_INDEX =comorbidity_data_non_afib.groupby(['DIABETES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_AT_INDEX['percentage (%)'] = DIABETES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_AT_INDEX)

print('=======================================')
DYSLIPIDEMIA_AT_INDEX =comorbidity_data_non_afib.groupby(['DYSLIPIDEMIA_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_AT_INDEX['percentage (%)'] = DYSLIPIDEMIA_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_AT_INDEX)

print('=======================================')
CHF_AT_INDEX =comorbidity_data_non_afib.groupby(['CHF_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_AT_INDEX['percentage (%)'] = CHF_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_AT_INDEX)

print('=======================================')
HYPERCOAG_STATES_AT_INDEX =comorbidity_data_non_afib.groupby(['HYPERCOAG_STATES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_AT_INDEX['percentage (%)'] = HYPERCOAG_STATES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_AT_INDEX)

print('=======================================')
MILD_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib.groupby(['MILD_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_AT_INDEX['percentage (%)'] = MILD_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_AT_INDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib.groupby(['MOD_SEV_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_AT_INDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib.groupby(['CHRONIC_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LUNG_DIS_AT_INDEX =comorbidity_data_non_afib.groupby(['CHRONIC_LUNG_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LUNG_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_AT_INDEX)

print('=======================================')
RHEUM_DIS_AT_INDEX =comorbidity_data_non_afib.groupby(['RHEUM_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_AT_INDEX['percentage (%)'] = RHEUM_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_AT_INDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_AT_INDEX =comorbidity_data_non_afib.groupby(['CHRONIC_KIDNEY_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_AT_INDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_AT_INDEX)

print('=======================================')
NEOPLASM_AT_INDEX =comorbidity_data_non_afib.groupby(['NEOPLASM_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_AT_INDEX['percentage (%)'] = NEOPLASM_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_AT_INDEX)

print('=======================================')
PERI_VASC_DIS_AT_INDEX =comorbidity_data_non_afib.groupby(['PERI_VASC_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_AT_INDEX['percentage (%)'] = PERI_VASC_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_AT_INDEX)

print('=======================================')
PFO_AT_INDEX =comorbidity_data_non_afib.groupby(['PFO_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_AT_INDEX['percentage (%)'] = PFO_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_AT_INDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_AT_INDEX =comorbidity_data_non_afib.groupby(['PAST_ISCHEMIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_AT_INDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_AT_INDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_AT_INDEX =comorbidity_data_non_afib.groupby(['PAST_HEMORRHAGIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_AT_INDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_AT_INDEX)

   ATRIAL_FIB_AT_INDEX  Patients  percentage (%)
0                    0      6185          100.00
   ATRIAL_FLUTTER_AT_INDEX  Patients  percentage (%)
0                        0      6156           99.53
1                        1        29            0.47
   AFIB_FLUTTER_AT_INDEX  Patients  percentage (%)
0                      0      6156           99.53
1                      1        29            0.47
   HYPERTENSION_AT_INDEX  Patients  percentage (%)
0                      0      1751           28.31
1                      1      4434           71.69
   MI_AT_INDEX  Patients  percentage (%)
0            0      5565           89.98
1            1       620           10.02
   DIABETES_AT_INDEX  Patients  percentage (%)
0                  0      4252           68.75
1                  1      1933           31.25
   DYSLIPIDEMIA_AT_INDEX  Patients  percentage (%)
0                      0      2440           39.45
1                      1      3745           60.55
   CHF_AT_INDEX  Pat

# having at least one follow up within 3 years and no afib

In [44]:
data_full_non_afib_patients = pd.read_csv('data_full_non_afib_patients.csv')

In [45]:
print(data_full_non_afib_patients['PT_ID'].nunique())
data_full_non_afib_patients['INDEX_DT'] = pd.to_datetime(data_full_non_afib_patients['INDEX_DT'])
data_full_non_afib_patients['ENC_DT'] = pd.to_datetime(data_full_non_afib_patients['ENC_DT'])
data_full_non_afib_patients['Difference'] = (data_full_non_afib_patients['ENC_DT'] - data_full_non_afib_patients['INDEX_DT'])
data_full_non_afib_patients['Difference_in_years'] = (data_full_non_afib_patients['ENC_DT'] - data_full_non_afib_patients['INDEX_DT'])/np.timedelta64(1,'Y')
data_full_non_afib_encounter_after_stroke = data_full_non_afib_patients[data_full_non_afib_patients['Difference_in_years'] > 0]
print(data_full_non_afib_encounter_after_stroke['PT_ID'].nunique())

6185
5789


In [46]:
data_full_non_afib_patients_no_afib_3_years_follow_up = data_full_non_afib_encounter_after_stroke[data_full_non_afib_encounter_after_stroke['Difference_in_years'] > 3]
print(data_full_non_afib_patients_no_afib_3_years_follow_up['PT_ID'].nunique())
geisinger_non_afib_panda_3_year_followup = geisinger_non_afib_panda[geisinger_non_afib_panda['PT_ID'].isin(data_full_non_afib_patients_no_afib_3_years_follow_up['PT_ID'].unique())]
geisinger_non_afib_panda_3_year_followup['PT_ID'].nunique()

2190


2190

In [47]:
comorbidity_data_non_afib_3_year_followup = geisinger_non_afib_panda_3_year_followup[['PT_ID','PT_SEX','PT_STATUS','FAM_HEART_HIST','FAM_STROKE_HIST','ATRIAL_FIB_PREINDEX','ATRIAL_FLUTTER_PREINDEX','AFIB_FLUTTER_PREINDEX','HYPERTENSION_PREINDEX','MI_PREINDEX','DIABETES_PREINDEX','DYSLIPIDEMIA_PREINDEX','CHF_PREINDEX','HYPERCOAG_STATES_PREINDEX','MILD_LIVER_DIS_PREINDEX','MOD_SEV_LIVER_DIS_PREINDEX','CHRONIC_LIVER_DIS_PREINDEX','CHRONIC_LUNG_DIS_PREINDEX','RHEUM_DIS_PREINDEX','CHRONIC_KIDNEY_DIS_PREINDEX','NEOPLASM_PREINDEX','PERI_VASC_DIS_PREINDEX','PFO_PREINDEX','PAST_ISCHEMIC_STROKE_PREINDEX','PAST_HEMORRHAGIC_STROKE_PREINDEX','ATRIAL_FIB_AT_INDEX','ATRIAL_FLUTTER_AT_INDEX','AFIB_FLUTTER_AT_INDEX','HYPERTENSION_AT_INDEX','MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX','HYPERCOAG_STATES_AT_INDEX','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX','CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX','CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX','PFO_AT_INDEX','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX']]
comorbidity_data_non_afib_3_year_followup['PT_ID'].nunique()

2190

### Sex and Family History

In [48]:
SEX =comorbidity_data_non_afib_3_year_followup.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
SEX['percentage (%)'] = SEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(SEX)

print('=======================================')
FAM_HEART_HIST =comorbidity_data_non_afib_3_year_followup.groupby(['FAM_HEART_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_HEART_HIST['percentage (%)'] = FAM_HEART_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_HEART_HIST)

print('=======================================')
FAM_STROKE_HIST =comorbidity_data_non_afib_3_year_followup.groupby(['FAM_STROKE_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_STROKE_HIST['percentage (%)'] = FAM_STROKE_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_STROKE_HIST)

   PT_SEX  Patients  percentage (%)
0  Female      1009           46.07
1    Male      1181           53.93
   FAM_HEART_HIST  Patients  percentage (%)
0               0      1227           56.03
1               1       963           43.97
   FAM_STROKE_HIST  Patients  percentage (%)
0                0      1836           83.84
1                1       354           16.16


### CoMorbidity PREINDEX

In [49]:
ATRIAL_FIB_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['ATRIAL_FIB_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_PREINDEX['percentage (%)'] = ATRIAL_FIB_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_PREINDEX)

print('=======================================')
ATRIAL_FLUTTER_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['ATRIAL_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_PREINDEX['percentage (%)'] = ATRIAL_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_PREINDEX)

print('=======================================')
AFIB_FLUTTER_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['AFIB_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_PREINDEX['percentage (%)'] = AFIB_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_PREINDEX)

print('=======================================')
HYPERTENSION_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['HYPERTENSION_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_PREINDEX['percentage (%)'] = HYPERTENSION_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_PREINDEX)

print('=======================================')
MI_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['MI_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_PREINDEX['percentage (%)'] = MI_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_PREINDEX)

print('=======================================')
DIABETES_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['DIABETES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_PREINDEX['percentage (%)'] = DIABETES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_PREINDEX)

print('=======================================')
DYSLIPIDEMIA_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['DYSLIPIDEMIA_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_PREINDEX['percentage (%)'] = DYSLIPIDEMIA_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_PREINDEX)

print('=======================================')
CHF_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHF_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_PREINDEX['percentage (%)'] = CHF_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_PREINDEX)

print('=======================================')
HYPERCOAG_STATES_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['HYPERCOAG_STATES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_PREINDEX['percentage (%)'] = HYPERCOAG_STATES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_PREINDEX)

print('=======================================')
MILD_LIVER_DIS_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['MILD_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_PREINDEX['percentage (%)'] = MILD_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_PREINDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['MOD_SEV_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_PREINDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LIVER_DIS_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHRONIC_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_PREINDEX['percentage (%)'] = CHRONIC_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LUNG_DIS_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHRONIC_LUNG_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_PREINDEX['percentage (%)'] = CHRONIC_LUNG_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_PREINDEX)

print('=======================================')
RHEUM_DIS_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['RHEUM_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_PREINDEX['percentage (%)'] = RHEUM_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_PREINDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHRONIC_KIDNEY_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_PREINDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_PREINDEX)

print('=======================================')
NEOPLASM_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['NEOPLASM_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_PREINDEX['percentage (%)'] = NEOPLASM_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_PREINDEX)

print('=======================================')
PERI_VASC_DIS_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PERI_VASC_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_PREINDEX['percentage (%)'] = PERI_VASC_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_PREINDEX)

print('=======================================')
PFO_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PFO_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_PREINDEX['percentage (%)'] = PFO_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_PREINDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PAST_ISCHEMIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_PREINDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_PREINDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PAST_HEMORRHAGIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_PREINDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_PREINDEX)

   ATRIAL_FIB_PREINDEX  Patients  percentage (%)
0                    0      2190          100.00
   ATRIAL_FLUTTER_PREINDEX  Patients  percentage (%)
0                        0      2189           99.95
1                        1         1            0.05
   AFIB_FLUTTER_PREINDEX  Patients  percentage (%)
0                      0      2189           99.95
1                      1         1            0.05
   HYPERTENSION_PREINDEX  Patients  percentage (%)
0                      0      1202           54.89
1                      1       988           45.11
   MI_PREINDEX  Patients  percentage (%)
0            0      2061           94.11
1            1       129            5.89
   DIABETES_PREINDEX  Patients  percentage (%)
0                  0      1739           79.41
1                  1       451           20.59
   DYSLIPIDEMIA_PREINDEX  Patients  percentage (%)
0                      0      1307           59.68
1                      1       883           40.32
   CHF_PREINDEX  Pat

### CoMorbidity AT INDEX

In [50]:
ATRIAL_FIB_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['ATRIAL_FIB_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_AT_INDEX['percentage (%)'] = ATRIAL_FIB_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_AT_INDEX)

print('=======================================')
ATRIAL_FLUTTER_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['ATRIAL_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_AT_INDEX['percentage (%)'] = ATRIAL_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_AT_INDEX)

print('=======================================')
AFIB_FLUTTER_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['AFIB_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_AT_INDEX['percentage (%)'] = AFIB_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_AT_INDEX)

print('=======================================')
HYPERTENSION_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['HYPERTENSION_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_AT_INDEX['percentage (%)'] = HYPERTENSION_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_AT_INDEX)

print('=======================================')
MI_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['MI_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_AT_INDEX['percentage (%)'] = MI_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_AT_INDEX)

print('=======================================')
DIABETES_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['DIABETES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_AT_INDEX['percentage (%)'] = DIABETES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_AT_INDEX)

print('=======================================')
DYSLIPIDEMIA_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['DYSLIPIDEMIA_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_AT_INDEX['percentage (%)'] = DYSLIPIDEMIA_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_AT_INDEX)

print('=======================================')
CHF_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHF_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_AT_INDEX['percentage (%)'] = CHF_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_AT_INDEX)

print('=======================================')
HYPERCOAG_STATES_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['HYPERCOAG_STATES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_AT_INDEX['percentage (%)'] = HYPERCOAG_STATES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_AT_INDEX)

print('=======================================')
MILD_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['MILD_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_AT_INDEX['percentage (%)'] = MILD_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_AT_INDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['MOD_SEV_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_AT_INDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHRONIC_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LUNG_DIS_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHRONIC_LUNG_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LUNG_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_AT_INDEX)

print('=======================================')
RHEUM_DIS_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['RHEUM_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_AT_INDEX['percentage (%)'] = RHEUM_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_AT_INDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHRONIC_KIDNEY_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_AT_INDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_AT_INDEX)

print('=======================================')
NEOPLASM_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['NEOPLASM_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_AT_INDEX['percentage (%)'] = NEOPLASM_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_AT_INDEX)

print('=======================================')
PERI_VASC_DIS_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PERI_VASC_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_AT_INDEX['percentage (%)'] = PERI_VASC_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_AT_INDEX)

print('=======================================')
PFO_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PFO_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_AT_INDEX['percentage (%)'] = PFO_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_AT_INDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PAST_ISCHEMIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_AT_INDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_AT_INDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PAST_HEMORRHAGIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_AT_INDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_AT_INDEX)

   ATRIAL_FIB_AT_INDEX  Patients  percentage (%)
0                    0      2190          100.00
   ATRIAL_FLUTTER_AT_INDEX  Patients  percentage (%)
0                        0      2187           99.86
1                        1         3            0.14
   AFIB_FLUTTER_AT_INDEX  Patients  percentage (%)
0                      0      2187           99.86
1                      1         3            0.14
   HYPERTENSION_AT_INDEX  Patients  percentage (%)
0                      0       637           29.09
1                      1      1553           70.91
   MI_AT_INDEX  Patients  percentage (%)
0            0      1999           91.28
1            1       191            8.72
   DIABETES_AT_INDEX  Patients  percentage (%)
0                  0      1543           70.46
1                  1       647           29.54
   DYSLIPIDEMIA_AT_INDEX  Patients  percentage (%)
0                      0       823           37.58
1                      1      1367           62.42
   CHF_AT_INDEX  Pat

# having at least one follow up within 5 years and no afib

In [51]:
data_full_non_afib_patients_no_afib_5_years_follow_up = data_full_non_afib_encounter_after_stroke[data_full_non_afib_encounter_after_stroke['Difference_in_years'] > 5]
print(data_full_non_afib_patients_no_afib_5_years_follow_up['PT_ID'].nunique())
geisinger_non_afib_panda_5_year_followup = geisinger_non_afib_panda[geisinger_non_afib_panda['PT_ID'].isin(data_full_non_afib_patients_no_afib_5_years_follow_up['PT_ID'].unique())]
geisinger_non_afib_panda_5_year_followup['PT_ID'].nunique()

1309


1309

In [52]:
comorbidity_data_non_afib_5_year_followup = geisinger_non_afib_panda_5_year_followup[['PT_ID','PT_SEX','PT_STATUS','FAM_HEART_HIST','FAM_STROKE_HIST','ATRIAL_FIB_PREINDEX','ATRIAL_FLUTTER_PREINDEX','AFIB_FLUTTER_PREINDEX','HYPERTENSION_PREINDEX','MI_PREINDEX','DIABETES_PREINDEX','DYSLIPIDEMIA_PREINDEX','CHF_PREINDEX','HYPERCOAG_STATES_PREINDEX','MILD_LIVER_DIS_PREINDEX','MOD_SEV_LIVER_DIS_PREINDEX','CHRONIC_LIVER_DIS_PREINDEX','CHRONIC_LUNG_DIS_PREINDEX','RHEUM_DIS_PREINDEX','CHRONIC_KIDNEY_DIS_PREINDEX','NEOPLASM_PREINDEX','PERI_VASC_DIS_PREINDEX','PFO_PREINDEX','PAST_ISCHEMIC_STROKE_PREINDEX','PAST_HEMORRHAGIC_STROKE_PREINDEX','ATRIAL_FIB_AT_INDEX','ATRIAL_FLUTTER_AT_INDEX','AFIB_FLUTTER_AT_INDEX','HYPERTENSION_AT_INDEX','MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX','HYPERCOAG_STATES_AT_INDEX','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX','CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX','CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX','PFO_AT_INDEX','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX']]
comorbidity_data_non_afib_5_year_followup['PT_ID'].nunique()

1309

### Sex and Family History

In [53]:
SEX =comorbidity_data_non_afib_5_year_followup.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
SEX['percentage (%)'] = SEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(SEX)

print('=======================================')
FAM_HEART_HIST =comorbidity_data_non_afib_5_year_followup.groupby(['FAM_HEART_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_HEART_HIST['percentage (%)'] = FAM_HEART_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_HEART_HIST)

print('=======================================')
FAM_STROKE_HIST =comorbidity_data_non_afib_5_year_followup.groupby(['FAM_STROKE_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_STROKE_HIST['percentage (%)'] = FAM_STROKE_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_STROKE_HIST)

   PT_SEX  Patients  percentage (%)
0  Female       617           47.14
1    Male       692           52.86
   FAM_HEART_HIST  Patients  percentage (%)
0               0       711           54.32
1               1       598           45.68
   FAM_STROKE_HIST  Patients  percentage (%)
0                0      1102           84.19
1                1       207           15.81


### CoMorbidity PREINDEX

In [54]:
ATRIAL_FIB_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['ATRIAL_FIB_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_PREINDEX['percentage (%)'] = ATRIAL_FIB_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_PREINDEX)

print('=======================================')
ATRIAL_FLUTTER_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['ATRIAL_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_PREINDEX['percentage (%)'] = ATRIAL_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_PREINDEX)

print('=======================================')
AFIB_FLUTTER_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['AFIB_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_PREINDEX['percentage (%)'] = AFIB_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_PREINDEX)

print('=======================================')
HYPERTENSION_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['HYPERTENSION_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_PREINDEX['percentage (%)'] = HYPERTENSION_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_PREINDEX)

print('=======================================')
MI_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['MI_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_PREINDEX['percentage (%)'] = MI_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_PREINDEX)

print('=======================================')
DIABETES_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['DIABETES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_PREINDEX['percentage (%)'] = DIABETES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_PREINDEX)

print('=======================================')
DYSLIPIDEMIA_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['DYSLIPIDEMIA_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_PREINDEX['percentage (%)'] = DYSLIPIDEMIA_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_PREINDEX)

print('=======================================')
CHF_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHF_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_PREINDEX['percentage (%)'] = CHF_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_PREINDEX)

print('=======================================')
HYPERCOAG_STATES_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['HYPERCOAG_STATES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_PREINDEX['percentage (%)'] = HYPERCOAG_STATES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_PREINDEX)

print('=======================================')
MILD_LIVER_DIS_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['MILD_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_PREINDEX['percentage (%)'] = MILD_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_PREINDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['MOD_SEV_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_PREINDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LIVER_DIS_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHRONIC_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_PREINDEX['percentage (%)'] = CHRONIC_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LUNG_DIS_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHRONIC_LUNG_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_PREINDEX['percentage (%)'] = CHRONIC_LUNG_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_PREINDEX)

print('=======================================')
RHEUM_DIS_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['RHEUM_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_PREINDEX['percentage (%)'] = RHEUM_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_PREINDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHRONIC_KIDNEY_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_PREINDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_PREINDEX)

print('=======================================')
NEOPLASM_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['NEOPLASM_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_PREINDEX['percentage (%)'] = NEOPLASM_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_PREINDEX)

print('=======================================')
PERI_VASC_DIS_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PERI_VASC_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_PREINDEX['percentage (%)'] = PERI_VASC_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_PREINDEX)

print('=======================================')
PFO_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PFO_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_PREINDEX['percentage (%)'] = PFO_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_PREINDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PAST_ISCHEMIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_PREINDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_PREINDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PAST_HEMORRHAGIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_PREINDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_PREINDEX)

   ATRIAL_FIB_PREINDEX  Patients  percentage (%)
0                    0      1309          100.00
   ATRIAL_FLUTTER_PREINDEX  Patients  percentage (%)
0                        0      1309          100.00
   AFIB_FLUTTER_PREINDEX  Patients  percentage (%)
0                      0      1309          100.00
   HYPERTENSION_PREINDEX  Patients  percentage (%)
0                      0       773           59.05
1                      1       536           40.95
   MI_PREINDEX  Patients  percentage (%)
0            0      1240           94.73
1            1        69            5.27
   DIABETES_PREINDEX  Patients  percentage (%)
0                  0      1072           81.89
1                  1       237           18.11
   DYSLIPIDEMIA_PREINDEX  Patients  percentage (%)
0                      0       830           63.41
1                      1       479           36.59
   CHF_PREINDEX  Patients  percentage (%)
0             0      1270           97.02
1             1        39            2.9

### CoMorbidity AT INDEX

In [55]:
ATRIAL_FIB_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['ATRIAL_FIB_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_AT_INDEX['percentage (%)'] = ATRIAL_FIB_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_AT_INDEX)

print('=======================================')
ATRIAL_FLUTTER_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['ATRIAL_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_AT_INDEX['percentage (%)'] = ATRIAL_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_AT_INDEX)

print('=======================================')
AFIB_FLUTTER_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['AFIB_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_AT_INDEX['percentage (%)'] = AFIB_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_AT_INDEX)

print('=======================================')
HYPERTENSION_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['HYPERTENSION_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_AT_INDEX['percentage (%)'] = HYPERTENSION_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_AT_INDEX)

print('=======================================')
MI_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['MI_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_AT_INDEX['percentage (%)'] = MI_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_AT_INDEX)

print('=======================================')
DIABETES_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['DIABETES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_AT_INDEX['percentage (%)'] = DIABETES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_AT_INDEX)

print('=======================================')
DYSLIPIDEMIA_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['DYSLIPIDEMIA_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_AT_INDEX['percentage (%)'] = DYSLIPIDEMIA_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_AT_INDEX)

print('=======================================')
CHF_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHF_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_AT_INDEX['percentage (%)'] = CHF_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_AT_INDEX)

print('=======================================')
HYPERCOAG_STATES_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['HYPERCOAG_STATES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_AT_INDEX['percentage (%)'] = HYPERCOAG_STATES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_AT_INDEX)

print('=======================================')
MILD_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['MILD_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_AT_INDEX['percentage (%)'] = MILD_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_AT_INDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['MOD_SEV_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_AT_INDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHRONIC_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LUNG_DIS_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHRONIC_LUNG_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LUNG_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_AT_INDEX)

print('=======================================')
RHEUM_DIS_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['RHEUM_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_AT_INDEX['percentage (%)'] = RHEUM_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_AT_INDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHRONIC_KIDNEY_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_AT_INDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_AT_INDEX)

print('=======================================')
NEOPLASM_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['NEOPLASM_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_AT_INDEX['percentage (%)'] = NEOPLASM_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_AT_INDEX)

print('=======================================')
PERI_VASC_DIS_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PERI_VASC_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_AT_INDEX['percentage (%)'] = PERI_VASC_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_AT_INDEX)

print('=======================================')
PFO_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PFO_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_AT_INDEX['percentage (%)'] = PFO_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_AT_INDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PAST_ISCHEMIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_AT_INDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_AT_INDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PAST_HEMORRHAGIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_AT_INDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_AT_INDEX)

   ATRIAL_FIB_AT_INDEX  Patients  percentage (%)
0                    0      1309          100.00
   ATRIAL_FLUTTER_AT_INDEX  Patients  percentage (%)
0                        0      1308           99.92
1                        1         1            0.08
   AFIB_FLUTTER_AT_INDEX  Patients  percentage (%)
0                      0      1308           99.92
1                      1         1            0.08
   HYPERTENSION_AT_INDEX  Patients  percentage (%)
0                      0       440           33.61
1                      1       869           66.39
   MI_AT_INDEX  Patients  percentage (%)
0            0      1201           91.75
1            1       108            8.25
   DIABETES_AT_INDEX  Patients  percentage (%)
0                  0       959           73.26
1                  1       350           26.74
   DYSLIPIDEMIA_AT_INDEX  Patients  percentage (%)
0                      0       534           40.79
1                      1       775           59.21
   CHF_AT_INDEX  Pat

In [56]:
data_full_non_afib_encounter_after_stroke[(data_full_non_afib_encounter_after_stroke['Difference_in_years'] > 3)&(data_full_non_afib_encounter_after_stroke['Difference_in_years'] > 5)].nunique()

PT_ID                  1309
ENC_DT                 3567
ENC_TYPE                  4
ICD_CD                 7651
ICD_NM                 7252
INDEX_DT               1074
Difference             3406
Difference_in_years    3406
dtype: int64